<a href="https://colab.research.google.com/github/thePhukan/Xception_EEG-/blob/parent/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 24 18:41:11 2021

@author: AtheP
"""
import tensorflow as tf


from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt

import glob
import PIL
from PIL import Image
import pandas as pd
from skimage.io import imread, imshow
import PIL
from PIL import Image

import os
from sklearn.model_selection import train_test_split, cross_val_score

dataset = pd.read_csv("/content/drive/MyDrive/M.Tech Project/DATASET/emotions_JUMBO.csv")
print("Any missing sample in training set:",dataset.isnull().values.any())

x = dataset.iloc[:,:-1].values

y = dataset.iloc[:,4908].values

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1, random_state=42,stratify = y)


In [ ]:


# Transforming non numerical labels into numerical labels
from sklearn import preprocessing


encoder = preprocessing.LabelEncoder()

# encoding train labels
encoder.fit(y_train)
Y_train = encoder.transform(y_train)

# encoding test labels
encoder.fit(y_test)
Y_test = encoder.transform(y_test)


params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Performing CV to tune parameters for best SVM fit
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(x_train, Y_train)



In [ ]:

# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n")

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n")
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(x_test)
Y_pred_label = list(encoder.inverse_transform(Y_pred))


# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(y_test,Y_pred_label))
print("\n")
print(classification_report(y_test,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(x_train , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(x_test  , Y_test ))
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precission: "+str(precision_score(y_true=y_test, y_pred=Y_pred,average="weighted")))
print("Recall: "+str(recall_score(y_true=y_test, y_pred=Y_pred,average="weighted")))
print("F1: "+str(f1_score(y_true=y_test, y_pred=Y_pred,average="weighted")))
scores_cv = cross_val_score(final_model, x_train, Y_train, cv=5)
# print(scores_cv)
print("Mean:",scores_cv.mean())
print("Standard deviation:",scores_cv.std())
print("cross vlaidation score: "+str(scores_cv.mean())+"+/-"+str(scores_cv.std()))
svm_model.score
print("SVM Score:",svm_model.score)